<h1>Import data</h1>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # autre compte gmail
# !cp '/content/drive/MyDrive/cassava/kaggle.json' '/content'

In [ ]:
# # compte ammi
# !cp '/content/drive/MyDrive/Cassava/kaggle.json' '/content'

In [ ]:
# ! pip install kaggle

In [ ]:
# import zipfile
# import os

# os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
# !kaggle competitions download -c ammi-2021-convnets

In [ ]:
# ! mkdir paddy
# ! unzip ammi-2021-convnets -d paddy
# # ! rm -R ammi-2021-convnets.zip

In [2]:
!pip install d2l==0.17.5

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision # load datasets
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import KFold
import albumentations as A
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader
import cv2
cudnn.benchmark = True
from PIL import Image
import glob
import os 
import seaborn as sns

# # Batch size during training
batch_size = 16
image_size = 325
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
data_dir = '../input/paddy-disease-classification/train_images'
test_data_dir = '../input/paddy-disease-classification/test_images'

In [4]:
train_transforms = transforms.Compose([
                              transforms.RandomRotation(30),
                              transforms.Resize(360),
                              transforms.RandomCrop(image_size),
                              transforms.RandomHorizontalFlip(0.3),
                              transforms.RandomVerticalFlip(0.3),
                              transforms.ToTensor(),
                              transforms.RandomErasing(0.1),
                              transforms.Normalize(mean=mean, std=std),
                          ])
test_transforms =transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize(mean=mean, std=std),
])
train_data = datasets.ImageFolder(root=data_dir ,  transform=train_transforms)   


In [5]:
classes = train_data.classes
classes

In [6]:
n = len(train_data)  # total number of examples
n_test = int(0.15 * n)  # take ~10% for test

test_set = torch.utils.data.Subset(train_data, range(n_test))  # take first 10%
train_set = torch.utils.data.Subset(train_data, range(n_test, n))  # take the rest   
len(train_set), len(test_set)

In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True,)
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

len(train_loader), len(test_loader)

In [8]:
from collections import Counter
# print(train_data.targets)
res = dict(Counter(train_data.targets))
print(res)
for key in res.keys():
  print(classes[key], res[key])

# Datetime month countplot
# plt.figure(figsize = (14, 7))
# sns.countplot(x = 'classes', data = train_data.targets)
# plt.title('Datetime month count plot')
# plt.show()

In [9]:
def imshow(img):
  ''' function to show image '''
  img = img / 2 + 0.5 # unnormalize
  npimg = img.numpy() # convert to numpy objects
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

In [10]:
dataiter = iter(train_loader)
features, labels = dataiter.next()
# call function on our images
print(features.shape)
print(labels)
imshow(torchvision.utils.make_grid(features))
# print(labels)
# print the class of the image
print(' '.join('%s' % classes[labels[j]] for j in range(batch_size)))

In [11]:

stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([]); ax.set_yticks([])
        denorm_images = denormalize(images, *stats)
        ax.imshow(torchvision.utils.make_grid(denorm_images[:64], nrow=8).permute(1, 2, 0).clamp(0,1))
        break

show_batch(train_loader)

<h1>Preprocess data </h1>

<h1>Train and Test data </h1>

In [12]:
from d2l import torch as d2l
torch.cuda.empty_cache()
torch.manual_seed(17)

In [13]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [14]:
def train(model, criterion, data_loader, optimizer, num_epochs):
    """Simple training loop for a PyTorch model."""

    # Make sure model is in training mode.
    model.train()

    # Move model to the device (CPU or GPU).
    model.to(device)

    # Exponential moving average of the loss.
    ema_loss = None

    # Loop over epochs.
    for epoch in range(num_epochs):

        # Loop over data.
        for batch_idx, (data, target) in enumerate(data_loader):

            # Forward pass.
            output = model(data.to(device))
            loss = criterion(output.to(device), target.to(device))

            # Backward pass.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # NOTE: It is important to call .item() on the loss before summing.
            if ema_loss is None:
                ema_loss = loss.item()
            else:
                ema_loss += (loss.item() - ema_loss) * 0.01

        # Print out progress the end of epoch.
        print(
            "Train Epoch: {} \tLoss: {:.6f}".format(epoch, ema_loss),
        )


def test(model, data_loader):
    """Measures the accuracy of a model on a data set."""
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0

    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():

        # Loop over test data.
        for data, target in data_loader:

            # Forward pass.
            output = model(data.to(device))

            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)

            # Count number of correct predictions.
            correct += pred.cpu().eq(target.view_as(pred)).sum().item()

    # Print test accuracy.
    percent = 100.0 * correct / len(data_loader.dataset)
    print(f"Accuracy: {correct} / {len(data_loader.dataset)} ({percent:.0f}%)")
    return percent

Tuning


In [15]:
# If `param_group=True`, the model parameters in the output layer will be
# updated using a learning rate ten times greater
def train_fine_tuning(net, learning_rate, train_data, test_data, batch_size=16, num_epochs=10,
                      param_group=True):

    # n = len(train_data)  # total number of examples
    # n_test = int(0.005 * n)  # take ~10% for test
    # print(n_test)
    # test_set = torch.utils.data.Subset(train_data, range(n_test))  # take first 10%
    # train_set = torch.utils.data.Subset(train_data, range(n_test, n))  # take the rest   
    
    train_iter = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True,)
    test_iter = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

    devices = d2l.try_all_gpus()
    loss = nn.CrossEntropyLoss(reduction="none")
    # trainer = AdamHD(net.parameters(), lr= learning_rate, hypergrad_lr=1e-9)
    trainer = torch.optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-5)


    # if param_group:
    #     params_1x = [param for name, param in net.named_parameters()
    #         #  if name not in ["classifier.weight", "classifier.bias"]]
    #          if name not in ["fc.weight", "fc.bias"]]
             
    #     trainer = torch.optim.SGD([{'params': params_1x},
    #                                {'params': net.fc.parameters(),
    #                               #  {'params': net.classifier.parameters(),
    #                                 'lr': learning_rate * 10}],
    #                                 lr=learning_rate, weight_decay=1e-3, momentum=0.9)
    # else:
    #     trainer = torch.optim.SGD(net.parameters(), lr=learning_rate,
    #                               weight_decay=1e-3, momentum=0.9)
        # trainer = AdamHD(net.parameters(), lr= learning_rate, hypergrad_lr=1e-9)

    d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs,
                   devices)

<h2>Train with all the data </h2>


In [16]:
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

In [17]:
n = len(train_data) 
n_test = int(0.2 * n)  # take ~10% for test
print(n_test, n)
test_set = torch.utils.data.Subset(train_data, range(n_test))  # take first 10%

finetune_net = torchvision.models.resnet50(pretrained=True) #loss 0.340, train acc 0.885, test acc 0.140 #best 
# finetune_net = torchvision.models.resnext101_32x8d(pretrained=True) # 
train_fine_tuning(finetune_net, 1e-4, train_data, test_set, num_epochs= 10)

<h1>final<h1>


In [18]:
test_transforms =transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                              #  transforms.Normalize(mean=mean, std=std),
                              transforms.Normalize(mean=mean, std=std),
])
test_data_dir = '../input/paddy-disease-classification/train_images'
final_test_data = datasets.ImageFolder(root=test_data_dir ,  transform=test_transforms)  
final_test_loader = torch.utils.data.DataLoader(final_test_data, batch_size=batch_size)


In [19]:
def final_test(model, data_loader):
    """Measures the accuracy of a model on a data set."""
    # Make sure the model is in evaluation mode.

    model.eval()
    correct = 0
    result = None
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():

        # Loop over test data.
        for data, target in data_loader:
            # print(data.dataset)
            # Forward pass.
            output = model(data.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)

            if result is not None:
              result = torch.concat((result, pred), axis=0)
            else: 
              result = pred

    print(result.shape)
    return result 

In [21]:
# pred = final_test(model, final_test_loader)
# pred = final_test(conv_model, final_test_loader)
pred = final_test(finetune_net, final_test_loader)

columns = ['image_id','label']
df = pd.DataFrame(columns=columns)

df['label'] = final_test_loader.dataset.samples
df['image_id'] = pred.squeeze().detach().cpu().numpy()

df['label'] = df['label'].apply(lambda x: x[0].split('/')[-1])
df['image_id'] = df['image_id'].apply(lambda x: classes[x])

df['sort'] = df['label'].str.extract('(\d+)', expand=False).astype(int)
df.sort_values('sort',inplace=True, ascending=True)
df = df.drop('sort', axis=1)
df.to_csv('Baseline1.csv', index = False)